In [ ]:
import arviz as az
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pymc as pm
import numpy as np
from scipy import stats

In [ ]:
df = pd.read_pickle('./data/processed/counts_df.pkl')
df.head()

In [ ]:
df['name_concat'] = df['name'].apply( lambda x: ' '.join([y for y in x if y is not None]) )
df['year_after_2011'] = df.year - 2011
df.head()

## Exploratory Data Analysis

### Capacity (MWe)

In [ ]:
fig = px.histogram(df['production (MWe)'])
fig.update_layout(
    width=750,
    height=300,
    template='plotly_white',
    xaxis_title='Production (MWe)',
    yaxis_title='Count',
    showlegend=False,
    font_size=18,
)
fig.show()

### Scrams

In [ ]:
# Actual Counts
hist_x, hist_y = [], []
for cnt in range(0, max(df['scrams']) + 1):
    hist_x.append(cnt)
    hist_y.append(len(df[df['scrams'] == cnt]))
hist_x, hist_y

# Theoretical Counts
lambda_ = np.mean(df['scrams'])
theory_y = [stats.poisson.pmf(x, mu=lambda_)*len(df) for x in hist_x]

# Figure
fig = go.Figure()
fig.add_trace(
    go.Bar(x=hist_x, y=hist_y, name='Empirical')
)
fig.add_trace(
    go.Bar(x=hist_x, y=theory_y, name=f'Poisson({lambda_:.2f})')
)
fig.update_layout(
    width=750,
    height=400,
    template='plotly_white',
    xaxis_title='Scrams',
    yaxis_title='Count',
    legend=dict(
        yanchor="top",
        y=1.0,
        xanchor="left",
        x=0.65
    ),
    font_size=18,
)
fig.show()

## Model

In [ ]:
reactor_idxs, reactors = pd.factorize(df.name_concat)
coords = {
    "reactors": reactors,
    "obs_id": np.arange(len(reactor_idxs)),
}

In [ ]:
with pm.Model(coords=coords) as hierarchical_model:
    
    reactor_idx = pm.ConstantData("reactor_idx", reactor_idxs, dims="obs_id")
    
    # Priors for coefficients
    mu_b1 = pm.Normal("mu_b1", mu=0.0, sigma=0.2**2 )
    sigma_b1 = pm.InverseGamma("sigma_b1", alpha=1, beta=1)
    
    mu_b2 = pm.Normal("mu_b2", mu=0.0, sigma=10**6)
    sigma_b2 = pm.InverseGamma("sigma_b2", alpha=1, beta=1)
    
    # Coefficients
    b_1 = pm.Normal("b1", mu=mu_b1, sigma=sigma_b1, dims="reactors")
    b_2 = pm.Normal("b2", mu=mu_b2, sigma=sigma_b2, dims="reactors")
    
    # - Zero-inflation Probability
    psi_est = 1 / ( 1 + pm.math.exp( -b_1[reactor_idx] ) )
    
    # - Poisson Rate
    mu_est = pm.math.exp( b_2[reactor_idx] )
    
    # Data Likelihood
    scram_likelihood = pm.ZeroInflatedPoisson(
        "scram_likelihood", psi=psi_est, mu=mu_est, observed=df.scrams.values, dims="obs_id",
    )
    
pm.model_to_graphviz(hierarchical_model)

In [ ]:
# Inference
with hierarchical_model:
    hierarchical_trace = pm.sample(
        10_000,
        tune=2_000,
        chains=4,
        target_accept=0.9,
    )

### Trace Plots

In [ ]:
az.plot_trace(hierarchical_trace, var_names=["mu_b1", "mu_b2"])

In [ ]:
az.plot_trace(hierarchical_trace, var_names=["sigma_b1", "sigma_b2"])

In [ ]:
az.plot_trace(hierarchical_trace, var_names=["b1", "b2"])

### Autocorrelation Plot

In [ ]:
az.plot_autocorr(hierarchical_trace, var_names=['mu_b1', 'mu_b2'], combined=True)

In [ ]:
az.plot_autocorr(hierarchical_trace, var_names=["sigma_b1", "sigma_b2"], combined=True)

## Results

In [ ]:
distributions / interpretations of mu_b1, mu_b2

means of various b1s, b2s -- show if some are much higher
- compare to reality